# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import sagemaker
from sagemaker import get_execution_role
import subprocess
import json





# magic word for producing visualizations in notebook
%matplotlib inline

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Setup Sagemaker Session
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
execution_role = sagemaker.session.get_execution_role()
region = sagemaker_session.boto_region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
#download data to notebook
#define data location constants
local_data_dir = 'data'
s3_data_path = f's3://{bucket}/data' 
s3_model_path = f's3://{bucket}/model'

## Initial Model and Kaggle Submission

Below I will be setting up the an initial AutoGluon run without any refienment of the data. Then I'll be submitting to Kaggle.

In [4]:
%%capture

!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0" bokeh==2.0.1
!pip install autogluon --no-cache-dir
!pip install kaggle
!pip install python-dotenv
from autogluon.tabular import TabularPredictor


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 205.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 126.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 193.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 241.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 284.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 284.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 160.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 251.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 228.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 170.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

### Setting up Kaggle Creds


In [12]:
!mkdir -p kaggle
%env KAGGLE_CONFIG_DIR=kaggle
!touch kaggle/kaggle.json
!chmod 600 kaggle/kaggle.json

env: KAGGLE_CONFIG_DIR=kaggle


In [21]:
from dotenv import dotenv_values

CONFIG = dotenv_values('env.txt')
kaggle_username = CONFIG['KAGGLE_USERNAME']
kaggle_key = CONFIG['KAGGLE_KEY']

# Save API token the kaggle.json file
with open("kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

### Downloading and Prepping Data

In [45]:
train_data = pd.read_csv(f'{s3_data_path}/train.csv')

/tmp/ipykernel_5821/3792393069.py:1: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(f'{s3_data_path}/train.csv')


In [46]:
train_data.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,1763,2,1.0,8.0,NaN,NaN,NaN,NaN,8.0,15.0,...,5.0,2.0,1.0,6.0,9.0,3.0,3,0,2,4
1,1771,1,4.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,1.0,2.0,1.0,4.0,9.0,7.0,1,0,2,3
2,1776,1,1.0,9.0,NaN,NaN,NaN,NaN,7.0,0.0,...,6.0,4.0,2.0,NaN,9.0,2.0,3,0,1,4
3,1460,2,1.0,6.0,NaN,NaN,NaN,NaN,6.0,4.0,...,8.0,11.0,11.0,6.0,9.0,1.0,3,0,2,4
4,1783,2,1.0,9.0,NaN,NaN,NaN,NaN,9.0,53.0,...,2.0,2.0,1.0,6.0,9.0,3.0,3,0,1,3


In [47]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42962 entries, 0 to 42961
Columns: 367 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(6)
memory usage: 120.3+ MB


### Doing Simple Training
To start for our first submission we're going to drop all the columns but the ones explored in the proposal section, this is simply to get a basic submission and baseline to compare against at the end of the project.

In [48]:
train_data[['RESPONSE', 'LNR', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'ANREDE_KZ']].describe()

,RESPONSE,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42962.000000,42962.000000,42357.000000,42357.000000,42357.000000,42962.000000
mean,0.012383,42803.120129,17.661071,5.274996,5.927001,1.595084
std,0.110589,24778.339984,14.085702,4.470538,3.398336,0.490881
min,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,0.000000,21284.250000,6.000000,2.000000,3.000000,1.000000
50%,0.000000,42710.000000,15.000000,4.000000,5.000000,2.000000
75%,0.000000,64340.500000,32.000000,10.000000,9.000000,2.000000
max,1.000000,85795.000000,40.000000,12.000000,10.000000,2.000000


In [49]:
selected_columns = ['RESPONSE', 'LNR', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'ANREDE_KZ']
init_train = train_data[selected_columns]

In [50]:
init_train.head()

,RESPONSE,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
0,0,1763,8.0,2.0,3.0,2
1,0,1771,19.0,5.0,9.0,2
2,0,1776,0.0,0.0,10.0,1
3,0,1460,16.0,4.0,3.0,2
4,0,1783,9.0,3.0,6.0,1


In [51]:
init_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42962 entries, 0 to 42961
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RESPONSE             42962 non-null  int64  
 1   LNR                  42962 non-null  int64  
 2   LP_LEBENSPHASE_FEIN  42357 non-null  float64
 3   LP_LEBENSPHASE_GROB  42357 non-null  float64
 4   LP_STATUS_FEIN       42357 non-null  float64
 5   ANREDE_KZ            42962 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 2.0 MB


In [52]:
features = ['LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'ANREDE_KZ']
init_train = init_train.dropna(subset=features).copy()

In [53]:
init_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42357 entries, 0 to 42961
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RESPONSE             42357 non-null  int64  
 1   LNR                  42357 non-null  int64  
 2   LP_LEBENSPHASE_FEIN  42357 non-null  float64
 3   LP_LEBENSPHASE_GROB  42357 non-null  float64
 4   LP_STATUS_FEIN       42357 non-null  float64
 5   ANREDE_KZ            42357 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 2.3 MB


In [54]:
# Normalize key values
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
columns_to_normalize = ['LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN']
init_train[columns_to_normalize] = scaler.fit_transform(init_train[columns_to_normalize])


In [55]:
init_train.describe()

,RESPONSE,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42357.000000,42357.000000,4.235700e+04,4.235700e+04,4.235700e+04,42357.000000
mean,0.012442,42801.576764,-1.087026e-16,6.106135e-17,4.579601e-17,1.595510
std,0.110848,24780.834820,1.000012e+00,1.000012e+00,1.000012e+00,0.490799
min,0.000000,1.000000,-1.253845e+00,-1.179960e+00,-1.449845e+00,1.000000
25%,0.000000,21275.000000,-8.278756e-01,-7.325817e-01,-8.613144e-01,1.000000
50%,0.000000,42709.000000,-1.889223e-01,-2.852030e-01,-2.727842e-01,2.000000
75%,0.000000,64347.000000,1.017990e+00,1.056933e+00,9.042763e-01,2.000000
max,1.000000,85795.000000,1.585948e+00,1.504312e+00,1.198541e+00,2.000000


In [56]:
#categorize sex
init_train['ANREDE_KZ'] = init_train['ANREDE_KZ'].map({1: 'Male', 2: 'Female'}).astype('category')

In [57]:
init_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42357 entries, 0 to 42961
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   RESPONSE             42357 non-null  int64   
 1   LNR                  42357 non-null  int64   
 2   LP_LEBENSPHASE_FEIN  42357 non-null  float64 
 3   LP_LEBENSPHASE_GROB  42357 non-null  float64 
 4   LP_STATUS_FEIN       42357 non-null  float64 
 5   ANREDE_KZ            42357 non-null  category
dtypes: category(1), float64(3), int64(2)
memory usage: 2.0 MB


In [58]:
init_train.head()

,RESPONSE,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
0,0,1763,-0.685886,-0.732582,-0.861314,Female
1,0,1771,0.095057,-0.061514,0.904276,Female
2,0,1776,-1.253845,-1.179960,1.198541,Male
3,0,1460,-0.117927,-0.285203,-0.861314,Female
4,0,1783,-0.614891,-0.508892,0.021481,Male


In [59]:
init_train.set_index('LNR', inplace=True)
predictor = TabularPredictor(label="RESPONSE", eval_metric='log_loss').fit(
    train_data=init_train,
    time_limit=600,
    presets="best_quality"
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231130_140410"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20231130_140410/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.16.41.198, ID: 6c4c5d6a5a83e73b75a2d41da1b682098f8c4f7afb0ff6fcd1425029) where the task (task ID: 29bc9987691ae2e47fd79286bed5c45830c8ef1d01000000, name=_sub_fit, pid=22103, memory used=0.43GB) was running was 3.57GB / 3.76GB (0.950027), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: a60f430bacbaed466b0240e1c016508befed931f29a0aab2f9aed63e) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.16.41.198`. To see the logs of the worker, use `ray logs worker-a60f430bacbaed466b0240e1c016508befed931f29a0aab2f9aed63e*out -ip 172.16.41.198. Top 10 memory users:
PID	MEM(GB)	COMMAND
5821	1.82	/home/ec2-user/anaconda3/envs/python3/bin/python -m ipykernel_launcher -f /home/ec2-user/.local/shar...
22103	0.43	ray::_sub_fit
10860	0.16	/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/

In [19]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2  -0.066645    log_loss       3.607294  299.268882                0.003067           4.502586            2       True         13
1    NeuralNetFastAI_BAG_L1  -0.066768    log_loss       1.335047  244.308689                1.335047         244.308689            1       True         10
2         LightGBMXT_BAG_L1  -0.066780    log_loss       0.175785   25.935876                0.175785          25.935876            1       True          3
3           LightGBM_BAG_L1  -0.066829    log_loss       0.050700   24.467825                0.050700          24.467825            1       True          4
4      LightGBMLarge_BAG_L1  -0.066842    log_loss       0.067685   24.387013                0.067685          24.387013            1       True         12
5 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': -0.19939988684776724,
  'KNeighborsDist_BAG_L1': -0.19938935052767662,
  'LightGBMXT_BAG_L1': -0.06677984446545061,
  'LightGBM_BAG_L1': -0.06682867410013044,
  'RandomForestGini_BAG_L1': -0.07491378454467366,
  'RandomForestEnt

In [18]:
model_path = predictor.path
print(f"The model was saved in: {model_path}")

The model was saved in: AutogluonModels/ag-20231130_133606


In [21]:
## Make copy of test file for submission
test_data = pd.read_csv(f'{s3_data_path}/test.csv')

/tmp/ipykernel_5821/9532272.py:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(f'{s3_data_path}/test.csv')


In [22]:
init_test_columns = [col for col in selected_columns if col != 'RESPONSE']
init_test = test_data[init_test_columns]
init_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42833 entries, 0 to 42832
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LNR                  42833 non-null  int64  
 1   LP_LEBENSPHASE_FEIN  42255 non-null  float64
 2   LP_LEBENSPHASE_GROB  42255 non-null  float64
 3   LP_STATUS_FEIN       42255 non-null  float64
 4   ANREDE_KZ            42833 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 1.6 MB


In [23]:
init_test.describe()

,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
count,42833.000000,42255.000000,42255.000000,42255.000000,42833.000000
mean,42993.165620,17.775175,5.313194,5.936528,1.595475
std,24755.599728,14.096616,4.475535,3.412463,0.490806
min,2.000000,0.000000,0.000000,1.000000,1.000000
25%,21650.000000,6.000000,2.000000,3.000000,1.000000
50%,43054.000000,15.000000,4.000000,5.000000,2.000000
75%,64352.000000,32.000000,10.000000,9.000000,2.000000
max,85794.000000,40.000000,12.000000,10.000000,2.000000


In [24]:
## Need to fill in NA values
init_test['LP_LEBENSPHASE_FEIN'].fillna(init_test['LP_LEBENSPHASE_FEIN'].mean(), inplace=True)
init_test['LP_LEBENSPHASE_GROB'].fillna(init_test['LP_LEBENSPHASE_GROB'].mean(), inplace=True)
init_test['LP_STATUS_FEIN'].fillna(init_test['LP_STATUS_FEIN'].mean(), inplace=True)
init_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42833 entries, 0 to 42832
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LNR                  42833 non-null  int64  
 1   LP_LEBENSPHASE_FEIN  42833 non-null  float64
 2   LP_LEBENSPHASE_GROB  42833 non-null  float64
 3   LP_STATUS_FEIN       42833 non-null  float64
 4   ANREDE_KZ            42833 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 1.6 MB


/tmp/ipykernel_5821/2243088850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_test['LP_LEBENSPHASE_FEIN'].fillna(init_test['LP_LEBENSPHASE_FEIN'].mean(), inplace=True)
/tmp/ipykernel_5821/2243088850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_test['LP_LEBENSPHASE_GROB'].fillna(init_test['LP_LEBENSPHASE_GROB'].mean(), inplace=True)
/tmp/ipykernel_5821/2243088850.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [26]:
# Use the same scaler from training data
init_test[columns_to_normalize] = scaler.transform(init_test[columns_to_normalize])  

/tmp/ipykernel_5821/234124756.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_test[columns_to_normalize] = scaler.transform(init_test[columns_to_normalize])


In [31]:
init_test.describe()

,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN
count,42833.000000,42833.000000,42833.000000,42833.000000
mean,42993.165620,0.008101,0.008544,0.002803
std,24755.599728,0.994011,0.994352,0.997370
min,2.000000,-1.253845,-1.179960,-1.449845
25%,21650.000000,-0.827876,-0.732582,-0.861314
50%,43054.000000,-0.188922,-0.285203,0.002803
75%,64352.000000,1.017990,1.056933,0.904276
max,85794.000000,1.585948,1.504312,1.198541


In [32]:
#categorize the same as training data
init_test['ANREDE_KZ'] = init_test['ANREDE_KZ'].map({1: 'Male', 2: 'Female'}).astype('category')  # Use the same mapping as in training data

/tmp/ipykernel_5821/3592771985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_test['ANREDE_KZ'] = init_test['ANREDE_KZ'].map({1: 'Male', 2: 'Female'}).astype('category')  # Use the same mapping as in training data


In [33]:
init_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42833 entries, 0 to 42832
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LNR                  42833 non-null  int64   
 1   LP_LEBENSPHASE_FEIN  42833 non-null  float64 
 2   LP_LEBENSPHASE_GROB  42833 non-null  float64 
 3   LP_STATUS_FEIN       42833 non-null  float64 
 4   ANREDE_KZ            0 non-null      category
dtypes: category(1), float64(3), int64(1)
memory usage: 1.3 MB


In [35]:
init_test.head()

,LNR,LP_LEBENSPHASE_FEIN,LP_LEBENSPHASE_GROB,LP_STATUS_FEIN,ANREDE_KZ
0,1754,0.166052,-0.061514,1.198541,NaN
1,1770,-0.827876,-0.732582,-1.449845,NaN
2,1465,1.585948,1.504312,1.198541,NaN
3,1470,-1.253845,-1.179960,-0.861314,NaN
4,1478,1.372964,1.504312,0.904276,NaN


In [34]:
predictions = predictor.predict(init_test)
predictions.describe()

count    42833.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: RESPONSE, dtype: float64

In [21]:
predictions.describe()

count    42833.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: RESPONSE, dtype: float64